###Coffee Leaf Disease Classification - Optimizer Comparison

Author: Tigist Wondimneh & Nahom Senay

ID: GSR/5506/17 and GSR/4848/17

### Importing libraries

In [48]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import numpy as np
import torch.nn.functional as F
from sklearn.metrics import f1_score, accuracy_score

In [49]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

### Load Dataset From Torch Vision

In [50]:
# Define transformations to apply to the images
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [51]:
# Pascal VOC 2012 has 20 classes
VOC_CLASSES = [
    'aeroplane', 'bicycle', 'boat','bus', 'car', 'motorbike', 'train'
]
EPOCHES = 100

In [52]:
class VOCDataset(torchvision.datasets.VOCDetection):
    def __getitem__(self, index):
        img, target = super().__getitem__(index)
        labels = [0] * len(VOC_CLASSES)
        objects = target['annotation']['object']
        if isinstance(objects, dict):
            objects = [objects]
        for obj in objects:
            cls = obj['name']
            if cls in VOC_CLASSES:
                labels[VOC_CLASSES.index(cls)] = 1
        return img, torch.FloatTensor(labels)

In [53]:
# Load datasets
train_dataset = VOCDataset(root='./data', year='2012', image_set='train', download=True, transform=transform)
val_dataset = VOCDataset(root='./data', year='2012', image_set='val', download=True, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

Using downloaded and verified file: ./data/VOCtrainval_11-May-2012.tar
Extracting ./data/VOCtrainval_11-May-2012.tar to ./data
Using downloaded and verified file: ./data/VOCtrainval_11-May-2012.tar
Extracting ./data/VOCtrainval_11-May-2012.tar to ./data


### Build A Simple CNN Model

In [ ]:
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, 3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        self.fc1 = nn.Linear(64 * 56 * 56, 256)
        self.fc2 = nn.Linear(256, len(VOC_CLASSES))

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        return torch.sigmoid(self.fc2(x))

### Define Optimizers

In [55]:
def get_basic_optimizers(model):
    return {
        # Part-I
        "Batch_Gradient_Descent": torch.optim.SGD(model.parameters(), lr=0.01),
        "Mini-batch SGD": torch.optim.SGD(model.parameters(), lr=0.01),
        "Stochastic Gradient Descent (SGD)": torch.optim.SGD(model.parameters(), lr=0.01),
    }

In [56]:

def get_advanced_optimizers(model):
    return {
        # # Part-II
        "Gradient Descent with Momentum": torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9),
        "Gradient Descent with Nesterov Momentum": torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9, nesterov=True),
    }

In [57]:

def get_adaptive_optimizers(model):
    return {
        # Part-III
        "AdaGrad": torch.optim.Adagrad(model.parameters(), lr=0.01),
        "RMSProp": torch.optim.RMSprop(model.parameters(), lr=0.001),
        "Adam": torch.optim.Adam(model.parameters(), lr=0.001)
    }

### Model Training and Evaluation

In [ ]:
def train(model, optimizer, train_loader, criterion, device):
    model.train()
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()


In [ ]:
def evaluate(model, val_loader, device):
    model.eval()
    y_true, y_pred = [], []
    with torch.no_grad():
        for images, labels in val_loader:
            images = images.to(device)
            outputs = model(images)
            preds = (outputs.cpu().numpy() > 0.5).astype(int)
            y_pred.extend(preds)
            y_true.extend(labels.numpy())
    acc = accuracy_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred, average='macro')
    return acc, f1


### Visualizations

In [60]:
def plot_results(results):
    names = list(results.keys())
    accs = [results[n]["Accuracy"] for n in names]
    f1s = [results[n]["F1 Score"] for n in names]

    x = np.arange(len(names))

    plt.figure(figsize=(12, 5))
    plt.bar(x, accs, width=0.6, color='skyblue')
    plt.ylabel('Accuracy')
    plt.title('Accuracy Comparison of Optimizers')
    plt.xticks(x, names, rotation=45, ha="right")
    plt.tight_layout()
    plt.show()

    plt.figure(figsize=(12, 5))
    plt.bar(x, f1s, width=0.6, color='lightgreen')
    plt.ylabel('F1 Score')
    plt.title('F1 Score Comparison of Optimizers')
    plt.xticks(x, names, rotation=45, ha="right")
    plt.tight_layout()
    plt.show()


### Basic Optimizers

In [61]:
criterion = nn.BCELoss()
basic_optimizers_results = {}

In [ ]:
for name in get_basic_optimizers(SimpleCNN()).keys():
    print(f"Training with {name}...")
    model = SimpleCNN().to(device)
    optimizer = get_basic_optimizers(model)[name]

    # Select appropriate training loader for BGD, SGD
    if name == "Stochastic Gradient Descent (SGD)":
        temp_loader = DataLoader(train_dataset, batch_size=1, shuffle=True)
    elif name == "Batch Gradient Descent (GD)":
        temp_loader = DataLoader(train_dataset, batch_size=len(train_dataset), shuffle=True)
    else:
        temp_loader = train_loader

    for epoch in range(EPOCHES):
        train(model, optimizer, temp_loader, criterion, device)
    acc, f1 = evaluate(model, val_loader, device)
    basic_optimizers_results[name] = {"Accuracy": acc, "F1 Score": f1}

Training with Batch_Gradient_Descent...


In [ ]:
plot_results(basic_optimizers_results)

In [ ]:
# Rankin Basic optimizers
sorted_results = sorted(basic_optimizers_results.items(), key=lambda x: x[1]['F1 Score'], reverse=True)
print("\nBasic Optimizer Ranking (based on F1 Score):")
for rank, (name, metrics) in enumerate(sorted_results, 1):
    print(f"{rank}. {name} - Accuracy: {metrics['Accuracy']:.4f}, F1 Score: {metrics['F1 Score']:.4f}")


### Advanced Optimizers

In [ ]:
criterion = nn.BCELoss()
advanced_optimizers_results = {}

In [ ]:
for name in get_advanced_optimizers(SimpleCNN()).keys():
    print(f"Training with {name}...")
    model = SimpleCNN().to(device)
    optimizer = get_advanced_optimizers(model)[name]
    for epoch in range(EPOCHES):
        train(model, optimizer, train_loader, criterion, device)
    acc, f1 = evaluate(model, val_loader, device)
    advanced_optimizers_results[name] = {"Accuracy": acc, "F1 Score": f1}

In [ ]:
plot_results(advanced_optimizers_results)

In [ ]:
# Rankin Basic optimizers
adv_sorted_results = sorted(advanced_optimizers_results.items(), key=lambda x: x[1]['F1 Score'], reverse=True)
print("\nBasic Optimizer Ranking (based on F1 Score):")
for rank, (name, metrics) in enumerate(adv_sorted_results, 1):
    print(f"{rank}. {name} - Accuracy: {metrics['Accuracy']:.4f}, F1 Score: {metrics['F1 Score']:.4f}")


### Adaptive OPtimizers

In [ ]:
criterion = nn.BCELoss()
adaptive_optimizers_results = {}

In [ ]:
for name in adaptive_optimizers_results(SimpleCNN()).keys():
    print(f"Training with {name}...")
    model = SimpleCNN().to(device)
    optimizer = adaptive_optimizers_results(model)[name]
    for epoch in range(EPOCHES):
        train(model, optimizer, train_loader, criterion, device)
    acc, f1 = evaluate(model, val_loader, device)
    advanced_optimizers_results[name] = {"Accuracy": acc, "F1 Score": f1}

In [ ]:
plot_results(adaptive_optimizers_results)

In [ ]:
# Rankin Basic optimizers
adap_sorted_results = sorted(adaptive_optimizers_results.items(), key=lambda x: x[1]['F1 Score'], reverse=True)
print("\nBasic Optimizer Ranking (based on F1 Score):")
for rank, (name, metrics) in enumerate(adap_sorted_results, 1):
    print(f"{rank}. {name} - Accuracy: {metrics['Accuracy']:.4f}, F1 Score: {metrics['F1 Score']:.4f}")


### Comparing the top 3 optimizers